In [85]:
"""
    Maps Standings and Fixtures for building the Training Data
"""

'\n    Maps Standings and Fixtures for building the Training Data\n'

In [1]:
import json
import os
import pandas as pd
from pandas import json_normalize

In [2]:
# Historicity: multi season directory
year_window = 1
start_season = 2020
seasons = list(range(start_season, start_season - year_window, -1))
print(seasons)

[2020]


In [3]:
# Multi League directory
country = 'MX'
dirCountry = '../' + country + '/'

In [4]:
league = '262'
season = '2020'

In [5]:
# League directory
dirName = dirCountry + league + '/'

In [6]:
# The last week played
curr_week = 10

In [8]:
df = pd.DataFrame()
standings = None
curr_standings = None
away_team_stands = None
fixture = None
# This will prevent duplicated fixtures
visited_fixtures = []
# This will help to map to standings to week: team : stands
visited_standings = {}
"""visited_standings['1'] = {}"""
# Easy access to last standings
current_standings = {}

# Principal leagues are sorted ascendent
dirLeague = sorted(os.listdir(dirCountry))

for league in dirLeague:
    if league == ".DS_Store":
        continue
    if "csv" in league:
        continue
    if ".json" in league:
        continue
    for season in seasons:
        dirName = dirCountry + league + '/' + str(season) + '/' 
        for subdir in os.listdir(dirName):
            if subdir == ".DS_Store":
                continue
            if subdir == "teams.json":
                continue
            if ".csv" in subdir:
                continue
            team_id = subdir
            # Entering directory of a team
            dirTeam = dirName + team_id + '/'
            
            # Initializes all first standings
            """
            if team_id not in visited_standings['1']:
                json_standings = dirTeam + team_id + '_' + str(1) + '.json'
                with open(json_standings) as f:
                    first_standings = json.load(f)
                    visited_standings['1'][team_id] = first_standings
            """
            
            # Initializes all current standings
            if team_id not in current_standings:
                json_standings = dirTeam + team_id + '_' + str(curr_week) + '.json'
                with open(json_standings) as f:
                    curr_standings = json.load(f)
                    current_standings[team_id] = curr_standings
            else:
                curr_standings = current_standings[team_id]
            
            # Gets standings from a team if they have not being added
            for week in list(range(1, 39, 1)):
                week = str(week)
                if week not in visited_standings:
                    visited_standings[week] = {}
                    if team_id not in visited_standings[week]:
                        try:
                            json_standings = dirTeam + team_id + '_' + week + '.json'
                            with open(json_standings) as f:
                                standings = json.load(f)
                                visited_standings[week][team_id] = standings
                        except:
                            visited_standings[week][team_id] = curr_standings
                            standings = curr_standings

            # Directory of team in home
            dirHome = dirTeam + 'home/'
            for file in os.listdir(dirHome):
                # Check if fixture hasn't being added to dataframe
                if file not in visited_fixtures:
                    path = dirHome + file
                    with open(path) as f:
                        fixture = json.load(f)
                        # Adds id of fixture as "id_fixture.json"
                        visited_fixtures.append(file)
                        # Next, get away team's external info
                        away_team = str(fixture["team_away"]["id"])
                        w = fixture["week"]
                        w = ''.join(filter(str.isdigit, w))
                        # Gets standings from away_team
                        """if w == '1':
                            try:
                                fileAwayTeam = dirName + away_team + '/' + away_team + '_' + str(1) + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings['1'][away_team] = away_team_stands
                            except:
                                print(fileAwayTeam)
                                print("Something rare")
                        """
                        if away_team in visited_standings[w]:
                            away_team_stands = visited_standings[w][away_team]
                        else:
                            try:
                                fileAwayTeam = dirName + away_team + '/' + away_team + '_' + week + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings[w][away_team] = away_team_stands
                            except:
                                fileAwayTeam = dirName + away_team + '/' + away_team + '_' + str(curr_week) + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    current_standings[away_team] = away_team_stands
                        # Home team standings
                        if team_id in visited_standings[w]:
                            standings = visited_standings[w][team_id]
                        else:
                            standings = current_standings[team_id]
                        # Gets away current standings if fixtures is set to null
                        if fixture["goals_away"] is None:
                            away_team_stands = current_standings[away_team]
                            # Changes if game to be played
                            standings = current_standings[team_id]
                        else:
                            if away_team in visited_standings[w]:
                                away_team_stands = visited_standings[w][away_team]
                            else:
                                away_team_stands = current_standings[away_team]
                    # Three main jsons to a dataframe
                    # standings "this is the home team"
                    # away_team_stands
                    # the fixture itself
                    # The premise that away standings from home team aren't relevant
                    try:
                        #del standings['stats_away']
                        fixture.update({"team_home": standings})
                    except:
                        print("Couldn't delete stats away")
                    # The premise that home standings from away team aren't relevant
                    try:
                        #del away_team_stands['stats_home']
                        fixture.update({"team_away": away_team_stands})
                    except:
                        print("Couldn't delete stats home")
                    # Adding league
                    fixture.update({"league": int(league)})
                    # Adding season
                    fixture.update({"season": season})
                    df = df.append(json_normalize(fixture))
                else:
                    continue

In [9]:
df.head(10)

,id,season,week,goals_home,goals_away,league,team_home.team.id,team_home.team.name,team_home.rank,team_home.points,...,stats_away.offside,stats_away.possession,stats_away.c_yellow,stats_away.c_red,stats_away.saves,stats_away.p_total,stats_away.p_accurate,stats_away.p_percentage,stats_home,stats_away
0,569686,2020,Apertura - 4,1,0,262,2288,Necaxa,16,4,...,2,63%,2,None,3,465.0,389.0,84%,NaN,NaN
0,569777,2020,Apertura - 14,None,None,262,2288,Necaxa,18,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
0,569724,2020,Apertura - 8,0,2,262,2288,Necaxa,15,8,...,1,69%,None,None,5,583.0,525.0,90%,NaN,NaN
0,569795,2020,Apertura - 16,None,None,262,2288,Necaxa,18,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
0,569679,2020,Apertura - 3,1,1,262,2288,Necaxa,18,1,...,None,43%,2,None,2,264.0,197.0,75%,NaN,NaN
0,569705,2020,Apertura - 6,2,1,262,2288,Necaxa,10,8,...,1,46%,None,None,3,258.0,181.0,70%,NaN,NaN
0,569740,2020,Apertura - 10,1,2,262,2288,Necaxa,18,8,...,1,57%,1,None,3,411.0,354.0,86%,NaN,NaN
0,570962,2020,Apertura - 1,0,3,262,2288,Necaxa,18,8,...,1,60%,1,None,1,547.0,474.0,87%,NaN,NaN
0,569749,2020,Apertura - 11,None,None,262,2288,Necaxa,18,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
0,569782,2020,Apertura - 14,None,None,262,2286,U.N.A.M. - Pumas,1,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [10]:
df.shape

(153, 124)

In [11]:
# csv to save
file_title = str(season) + '_' + str(curr_week) + '.csv'
csv_file = dirName + file_title
print(file_title)
print(csv_file)

2020_10.csv
../MX/262/2020/2020_10.csv


In [12]:
df.to_csv(csv_file)